Small demonstration how to add and query items.
The first part shows how to add by direct indexing, which is the most basic way to add stuff, also used in the internal machinery.
Then store.add does the same but only in one line. Lastly, store.add_all abstracts store.add to make it simple to add a batch of similar items with different combinations.

To retrieve items, store.get applies the same idea of API as store.add - a dictionary of predicate to object - in order to return the entities that match all criteria.
The result is a regular set, which makes it easy to combine with other queries.
To retrieve information about a given entity, just do store[entity] to get a namedtuple that represents that entity. If you don't know exactly what predicates an entity has before you query it, you can check the namedtuple.\_fields attribute.

A little heads up:
1. Keep in mind that store.add_all takes *lists* of things that will be iterated over. You can easily miss that if you just want to add a single predicate as string which is also an iterable.
2. store.get returns sets, which can't be indexed. If you don't need the set after, you can just set.pop(), but it can be an issue if you just want a single item without altering the set. The trick to do that (yes, believe it or not, this is the most efficient way) is
    
        s = {1,2,3}
        for x in s:
            break
        return x

In [6]:
from store.store import TripleStore, E, Predicate
    
body = TripleStore()

class Side(Predicate):
    pass

class Name(Predicate):
    pass

class Is_a(Predicate):
    pass

class Has(Predicate):
    pass

name = Name()
side = Side()
is_a = Is_a()
has = Has()


body.create_subjects_with({name:["prototype_sense"], is_a:["sense", "organ"], })
body.create_subjects_with({name:["eye"] side:["left", "right"], is_a:["organ", "sense"]})

fingers = "thumb index middle ring pinky"
body.add_all((Name(f) for f in fingers), [Side("left"), Side("right")], Is_a("finger"))

x = body.get_which(Is_a("organ")).pop()

SyntaxError: invalid syntax (<ipython-input-6-facbbef38c02>, line 24)

It is useful to add criteria for predicates, like types and possible ranges. This is done by store.set_check(p, func), which stores the function in a dict to be called when anything with that predicate is added. Most often, a simple lambda like

    store.set_check(P.number, lambda o: 1 < o < 10)
    
will do the trick.

## Sources

* [https://en.wikipedia.org/wiki/SPARQL]
* [http://www.eionet.europa.eu/software/sparql-client/]